In [4]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

BASE_URL = "https://rera.odisha.gov.in"
LIST_URL = BASE_URL + "/projects/project-list"

opts = Options()
opts.add_argument("--headless")
driver = webdriver.Chrome(options=opts)
wait = WebDriverWait(driver, 20)

rows = []

try:
    driver.get(LIST_URL)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card.project-card")))

    for idx in range(6):
        cards = driver.find_elements(By.CSS_SELECTOR, "div.card.project-card")
        card = cards[idx]

        btn = card.find_element(By.CSS_SELECTOR, "a.btn.btn-primary")
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
        time.sleep(0.3)
        driver.execute_script("arguments[0].click();", btn)

        wait.until(EC.url_contains("/projects/project-details/"))

        proj_name = wait.until(EC.visibility_of_element_located((
            By.XPATH, "//label[normalize-space(text())='Project Name']/following-sibling::strong"
        ))).text.strip()
        rera_no = driver.find_element(By.XPATH,
            "//label[normalize-space(text())='RERA Regd. No.']/following-sibling::strong"
        ).text.strip()

        promo_tab = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//a[normalize-space(text())='Promoter Details']"
        )))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", promo_tab)
        driver.execute_script("arguments[0].click();", promo_tab)

        wait.until(EC.presence_of_element_located((
            By.CSS_SELECTOR, "app-promoter-details label.label-control"
        )))

        promo = {}
        labels = driver.find_elements(By.CSS_SELECTOR, "app-promoter-details label.label-control")
        for label in labels:
            key = label.text.strip()
            try:
                val = label.find_element(
                    By.XPATH, "./following-sibling::strong[1]"
                ).text.strip()
            except:
                val = "--"
            promo[key] = val

        company = promo.get("Company Name", "--")
        gst_no = promo.get("GST No.", "--")
        reg_addr = promo.get("Registered Office Address", "--")

        rows.append([idx+1, proj_name, rera_no, company, gst_no, reg_addr])

        driver.back()
        wait.until(EC.url_contains("/projects/project-list"))
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card.project-card")))

finally:
    driver.quit()

with open("main_data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Idx",
        "Project Name",
        "RERA No.",
        "Company Name",
        "GST No.",
        "Registered Office Address"
    ])
    writer.writerows(rows)
